In [1]:
import pandas as pd

In [2]:
import findspark
findspark.init()
print findspark.find()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

/usr/local/opt/apache-spark/libexec


In [3]:
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
sqlsc=SQLContext(sc)

In [4]:
#rdd = sc.textFile('sacramentorealestatetransactions.csv')
rdd = sc.textFile('tweets.csv')



In [5]:
rdd.take(5)

[u'0,"{u\'type\': u\'Polygon\', u\'coordinates\': [[[-118.082615, 33.628991], [-118.082615, 33.756093], [-117.91485, 33.756093], [-117.91485, 33.628991]]]}",Huntington Beach,United States,In need of back rub,669375138903666692,1448426444160,1611125173',
 u'1,"{u\'type\': u\'Polygon\', u\'coordinates\': [[[-82.87387, 38.688052], [-82.87387, 38.76256], [-82.811927, 38.76256], [-82.811927, 38.688052]]]}",Wheelersburg,United States,And I\'m always tired but never of you,669375138958307328,1448426444173,3165522051',
 u'2,"{u\'type\': u\'Polygon\', u\'coordinates\': [[[-53.72354, -25.371418], [-53.72354, -24.750106], [-53.072456, -24.750106], [-53.072456, -25.371418]]]}",Cascavel,Brasil,@DudaDalmas eita,669375138555633664,1448426444077,171176445',
 u'3,"{u\'type\': u\'Polygon\', u\'coordinates\': [[[-117.457382, 33.966021], [-117.457382, 34.034229], [-117.367428, 34.034229], [-117.367428, 33.966021]]]}",Rubidoux,United States,"Lugo: I want to order pizza ',
 u"Me: let's do it "]

In [6]:
rdd = rdd.map(lambda line: line.split(","))
rdd.take(2)


[[u'0',
  u'"{u\'type\': u\'Polygon\'',
  u" u'coordinates': [[[-118.082615",
  u' 33.628991]',
  u' [-118.082615',
  u' 33.756093]',
  u' [-117.91485',
  u' 33.756093]',
  u' [-117.91485',
  u' 33.628991]]]}"',
  u'Huntington Beach',
  u'United States',
  u'In need of back rub',
  u'669375138903666692',
  u'1448426444160',
  u'1611125173'],
 [u'1',
  u'"{u\'type\': u\'Polygon\'',
  u" u'coordinates': [[[-82.87387",
  u' 38.688052]',
  u' [-82.87387',
  u' 38.76256]',
  u' [-82.811927',
  u' 38.76256]',
  u' [-82.811927',
  u' 38.688052]]]}"',
  u'Wheelersburg',
  u'United States',
  u"And I'm always tired but never of you",
  u'669375138958307328',
  u'1448426444173',
  u'3165522051']]

In [7]:
header = rdd.first()
rdd = rdd.filter(lambda line:line != header)
rdd.take(2)

[[u'1',
  u'"{u\'type\': u\'Polygon\'',
  u" u'coordinates': [[[-82.87387",
  u' 38.688052]',
  u' [-82.87387',
  u' 38.76256]',
  u' [-82.811927',
  u' 38.76256]',
  u' [-82.811927',
  u' 38.688052]]]}"',
  u'Wheelersburg',
  u'United States',
  u"And I'm always tired but never of you",
  u'669375138958307328',
  u'1448426444173',
  u'3165522051'],
 [u'2',
  u'"{u\'type\': u\'Polygon\'',
  u" u'coordinates': [[[-53.72354",
  u' -25.371418]',
  u' [-53.72354',
  u' -24.750106]',
  u' [-53.072456',
  u' -24.750106]',
  u' [-53.072456',
  u' -25.371418]]]}"',
  u'Cascavel',
  u'Brasil',
  u'@DudaDalmas eita',
  u'669375138555633664',
  u'1448426444077',
  u'171176445']]

In [21]:
df1 = rdd.map(lambda line: Row(bounding_box=line[1:10], city=line[10], country=line[11], 
                              text=line[12:-3], tid=line[-3], timestamp=line[-2], uid=line[-1])).toDF()
df1.count()

Py4JJavaError: An error occurred while calling o286.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 25, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-21-78d739051c95>", line 1, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRunner$$ano

In [9]:
df1 = rdd.map(lambda line: Row(bounding_box=line[1:10], city=line[10], country=line[11], 
                              text=line[12:-3], tid=line[-3], timestamp=line[-2], uid=line[-1])).toDF()
#df1 = rdd.map(lambda line: Row(street = line[0], city = line[1], 
#                              zip=line[2], beds=line[4], baths=line[5], sqft=line[6], price=line[9])).toDF()
print type(rdd)
print rdd.count()
print type(df1)


<class 'pyspark.rdd.PipelinedRDD'>
37518
<class 'pyspark.sql.dataframe.DataFrame'>


In [11]:
predf2 = rdd.map(lambda line: Row(bounding_box=line[1:10], city=line[10], country=line[11], 
                              text=line[12:-3], tid=line[-3], timestamp=line[-2], uid=line[-1]))
#predf2 = rdd.map(lambda line: Row(street = line[0], city = line[1], 
#                              zip=line[2], beds=line[4], baths=line[5], sqft=line[6], price=line[9]))
df2 = sqlsc.createDataFrame(predf2)
print type(rdd)
print rdd.count()
print type(df2)


<class 'pyspark.rdd.PipelinedRDD'>
37518
<class 'pyspark.sql.dataframe.DataFrame'>
